# 01_data_cleaning.ipynb

In [1]:
import pandas as pd
import klib
import sklearn
from sklearn.model_selection import train_test_split
from platform import python_version
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.precision',2)
print(f'Python {python_version()}, Pandas {pd.__version__}, sklearn{sklearn.__version__}, klib{klib.__version__}')

Python 3.9.7, Pandas 2.2.3, sklearn1.6.1, klib1.3.2


## 1.1 Load Raw Data
We first load the raw dataset, reset its index, and perform an initial inspection to understand the structure and distribution of the data.

### Read data, reset index and show basic info

In [2]:
data = pd.read_csv('../data/BankChurners.csv', index_col=0).iloc[:, :-2]
data.reset_index(drop=True, inplace=True)

print(data.info())
display(data.sample(5).T) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  object 
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  object 
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  object 
 5   Marital_Status            10127 non-null  object 
 6   Income_Category           10127 non-null  object 
 7   Card_Category             10127 non-null  object 
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

,5450,4653,2460,6922,9663
Attrition_Flag,Attrited Customer,Existing Customer,Existing Customer,Existing Customer,Existing Customer
Customer_Age,40,35,51,48,40
Gender,F,F,M,M,F
Dependent_count,2,1,1,5,3
Education_Level,High School,High School,Unknown,Unknown,College
Marital_Status,Single,Married,Married,Married,Single
Income_Category,Unknown,Less than $40K,$60K - $80K,$40K - $60K,Less than $40K
Card_Category,Blue,Blue,Blue,Blue,Silver
Months_on_book,30,26,36,38,36
Total_Relationship_Count,6,4,3,4,2


### Check values for each numeric features

In [3]:
data.describe()

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
count,10127.00,10127.00,10127.00,10127.00,10127.00,10127.00,10127.00,10127.00,10127.00,10127.00,10127.00,10127.00,10127.00,10127.00
mean,46.33,2.35,35.93,3.81,2.34,2.46,8631.95,1162.81,7469.14,0.76,4404.09,64.86,0.71,0.27
std,8.02,1.30,7.99,1.55,1.01,1.11,9088.78,814.99,9090.69,0.22,3397.13,23.47,0.24,0.28
min,26.00,0.00,13.00,1.00,0.00,0.00,1438.30,0.00,3.00,0.00,510.00,10.00,0.00,0.00
25%,41.00,1.00,31.00,3.00,2.00,2.00,2555.00,359.00,1324.50,0.63,2155.50,45.00,0.58,0.02
50%,46.00,2.00,36.00,4.00,2.00,2.00,4549.00,1276.00,3474.00,0.74,3899.00,67.00,0.70,0.18
75%,52.00,3.00,40.00,5.00,3.00,3.00,11067.50,1784.00,9859.00,0.86,4741.00,81.00,0.82,0.50
max,73.00,5.00,56.00,6.00,6.00,6.00,34516.00,2517.00,34516.00,3.40,18484.00,139.00,3.71,1.00


### Check values for each categorical features

In [4]:
cat_cols = data.select_dtypes(include=['object', 'category']).columns

for col in cat_cols:
    unique_vals = data[col].unique()
    print(f"Feature: {col}")
    print(f"Amount of unique values: {len(unique_vals)}")
    print(f"Unique values: {unique_vals}\n")

Feature: Attrition_Flag
Amount of unique values: 2
Unique values: ['Existing Customer' 'Attrited Customer']

Feature: Gender
Amount of unique values: 2
Unique values: ['M' 'F']

Feature: Education_Level
Amount of unique values: 7
Unique values: ['High School' 'Graduate' 'Uneducated' 'Unknown' 'College' 'Post-Graduate'
 'Doctorate']

Feature: Marital_Status
Amount of unique values: 4
Unique values: ['Married' 'Single' 'Unknown' 'Divorced']

Feature: Income_Category
Amount of unique values: 6
Unique values: ['$60K - $80K' 'Less than $40K' '$80K - $120K' '$40K - $60K' '$120K +'
 'Unknown']

Feature: Card_Category
Amount of unique values: 4
Unique values: ['Blue' 'Gold' 'Silver' 'Platinum']



## 1.2 Initial Inspection
We check for missing values across all features to determine if any imputation is necessary.

### Screening for missing values

In [5]:
klib.missingval_plot(data)

No missing values found in the dataset.


#### There are no missing values, no need to handling them

## 1.3 Feature Engineering
Several preprocessing steps are applied, including transforming the target variable and preparing feature and label datasets.

### Target Variable Transformation

To facilitate binary classification for predicting customer churn, we transformed the `Attrition_Flag` feature into a binary format:
- `0` representing "Existing Customer"
- `1` representing "Attrited Customer"
Then rename it to `Attrited_Customer`

In [6]:
data['Attrition_Flag'].replace({'Existing Customer':False, 'Attrited Customer':True}, inplace=True)

In [7]:
data.rename(columns={'Attrition_Flag':'Attrited_Customer'}, inplace=True)

In [8]:
data.shape

(10127, 20)

 ### Splitting Dataset into Training and Testing Sets

In [9]:
X = data.drop(columns=['Attrited_Customer'])
y = data['Attrited_Customer']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.2,stratify=y,random_state=6688)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8101, 19), (2026, 19), (8101,), (2026,))

## 1.4 Save Data for Further Use
### Saving Processed Data

The processed training and testing datasets are saved as separate CSV files for use in subsequent modeling steps.

In [10]:
X_train.merge(y_train, left_index=True, right_index=True).to_csv('../data/bank_churners_train.csv', index_label='index')
X_test.merge(y_test, left_index=True, right_index=True).to_csv('../data/bank_churners_test.csv', index_label='index')

## 1.5 Summary of Data Preparation

The dataset, sourced from Kaggle, includes 10,127 customer records and 20 features, spanning demographic information, credit card usage metrics, and account activity levels. No missing values were detected, and minimal cleaning was required. 

Key data preparation steps:
- Converted the `Attrition_Flag` into a binary target variable to facilitate churn prediction.
- Casted features into appropriate data types for analysis.
- Split the dataset into training and testing sets using stratified sampling to preserve class distribution.
- Saved the processed datasets for subsequent exploratory data analysis (EDA) and model development phases.

This preparation ensures the data is clean, well-structured, and ready for predictive modeling tasks.